In [1]:
from pathlib import Path
import polars as pl
import duckdb
from datasets import Dataset
import numpy as np
from autofaiss import build_index
from FlagEmbedding import BGEM3FlagModel
from core_pro.ultilities import make_dir, make_sync_folder


In [2]:
path = make_sync_folder("dataset/item_matching")
file = path / "data_sample_FMCG_clean.parquet"

query = f"""
select item_id
, item_name
from read_parquet('{file}')
limit 10000
"""
df = duckdb.sql(query).pl()
df.head()

item_id,item_name
i64,str
14019452100,"""Mắt Mèo Cầu Vồng Huaxi 7 Màu T…"
25423506264,"""*FREESHIP* Miếng Dán Trang Trí…"
25650112647,"""(Trụ vừa) Mi trụ katun, trụ wi…"
10158281345,"""Mascara Chuốt Mi JUDYDOLL 2g D…"
2486662254,"""Bột lắc sữa Bledina Pháp (Date…"


In [3]:
def create_search_result(file_embed, data):
    embeddings = np.load(file_embed)
    print(embeddings.shape)

    data = data.with_columns(pl.Series(values=embeddings, name="embed"))
    dataset = Dataset.from_polars(data)
    dataset.set_format(type="numpy", columns=["embed"], output_all_columns=True)

    path_index = Path(path / f"index_{file_embed.stem}")
    build_index(
        embeddings=embeddings,
        index_path=str(path_index),
        save_on_disk=True,
        metric_type="ip",
        verbose=30,
    )
    dataset.load_faiss_index("embed", path_index)

    score, result = dataset.get_nearest_examples_batch(
        "embed", dataset["embed"], k=5
    )
    for i in result:
        del i["embed"]

    dict_ = {"score": [list(i) for i in score]}
    df_score = pl.DataFrame(dict_)
    df_result = (
        pl.DataFrame(result).select(pl.all().name.prefix(f"db_"))
    )
    df_match = pl.concat([df, df_result, df_score], how="horizontal")
    return df_match

In [4]:
file_embed = path / "bi_encode.npy"
df_match = create_search_result(file_embed, df)

(10000, 768)


  0%|          | 0/1 [00:00<?, ?it/s]


In [5]:
file_embed = path / "bge_encode.npy"
df_match_bge = create_search_result(file_embed, df)

(10000, 1024)


  0%|          | 0/1 [00:00<?, ?it/s]


In [6]:
rename_cols = {"db_item_id": "db_bi_item_id", "db_item_name": "db_bi_item_name", "score": "score_bi"}
df_final = df.join(df_match.drop(["item_name"]).rename(rename_cols), how="left", on="item_id")

rename_cols = {"db_item_id": "db_bge_item_id", "db_item_name": "db_bge_item_name", "score": "score_bi"}
df_final = df_final.join(df_match_bge.drop(["item_name"]).rename(rename_cols), how="left", on="item_id")

In [7]:
dif = [set(i["db_bi_item_name"]) ^ set(i["db_bge_item_name"]) for i in df_final[["db_bi_item_name", "db_bge_item_name"]].to_dicts()]
df_final = df_final.with_columns(
    pl.Series("dif", dif),
    pl.Series("dif_len", [len(_) for _ in dif]),
)

In [8]:
df_final

item_id,item_name,db_bi_item_id,db_bi_item_name,score_bi,db_bge_item_id,db_bge_item_name,score_bi_right,dif,dif_len
i64,str,list[i64],list[str],list[f32],list[i64],list[str],list[f32],object,i64
14019452100,"""Mắt Mèo Cầu Vồng Huaxi 7 Màu T…","[14019452100, 29271686341, … 24039668933]","[""Mắt Mèo Cầu Vồng Huaxi 7 Màu Tạo Hiệu Ứng 9D ( Siêu Hót )"", ""Nấm Ngoài Da Cho Gà Đá , Chó ,Mèo Ketomycine Vàng Tuýp 25g"", … ""Nước Hoa Nữ Delina Exclusif Chính Hãng - Mùi Hương Ngọt Ngào, Uy Quyền, Sang Trọng""]","[0.999534, 0.735134, … 0.569959]","[14019452100, 29271686341, … 27400189915]","[""Mắt Mèo Cầu Vồng Huaxi 7 Màu Tạo Hiệu Ứng 9D ( Siêu Hót )"", ""Nấm Ngoài Da Cho Gà Đá , Chó ,Mèo Ketomycine Vàng Tuýp 25g"", … ""YADORNOS 24 Chiếc Thỏ Ngọt Hoa Camellia Kim Cương Giả Ngọc Trai Bé Gái Móng Tay Giả Nghệ Thuật""]","[1.000086, 0.744827, … 0.645257]","{'YADORNOS 24 Chiếc Thỏ Ngọt Hoa Camellia Kim Cương Giả Ngọc Trai Bé Gái Móng Tay Giả Nghệ Thuật', 'Thức Ăn Sấy Thú Cưng Mix Nhiều Vị - Dùng Được Cho Chó Và Mèo', 'Son dưỡng môi mỡ hươu Astrid Tái tạo môi khô Dưỡng môi mềm mại, Son Astrid', 'Phủ đen mi - lông mày, Thuốc nhuộm SABBUHA (3 màu: Nâu, Coffee, Đen) - Phan Dũng', 'Nước Hoa Nữ Delina Exclusif Chính Hãng - Mùi Hương Ngọt Ngào, Uy Quyền, Sang Trọng', 'Nailbox thiết kế c232 mẫu đơn giản chụp kỉ yếu đi học, đi chơi Cebi.nailbox'}",6
25423506264,"""*FREESHIP* Miếng Dán Trang Trí…","[25423506264, 20579970043, … 26817874290]","[""*FREESHIP* Miếng Dán Trang Trí Móng Tay Chống Thấm Nước Hoạt Hình Dễ Thương không thấm nước cho bé"", ""Xịt Dưỡng Tóc Hask Curl Care Argan oil 5-in-1 Leave-in Spray 175ml Phục Hồi Tóc Khỏi Hư Tổn Mềm Mượt Tóc Hàng Úc"", … ""Hộp 32 túi mù moi moi, rùa babythree, crybaby, mộng kuromi full box túi mù rùa moimoi quà tặng cho bé""]","[0.999482, 0.499663, … 0.429144]","[25423506264, 24970535248, … 17765522937]","[""*FREESHIP* Miếng Dán Trang Trí Móng Tay Chống Thấm Nước Hoạt Hình Dễ Thương không thấm nước cho bé"", ""Bộ 5 Đế Gắn Móng Quân Cờ Tập Vẽ Làm Nail | Đế Acrylic 5 Móng Nailbox Dễ Dàng Sử Dụng"", … ""[TẶNG MÚT CHE KHUYẾT ĐIỂM] MẪU MỚI - Kem Che Khuyết Điểm CKĐ The Saem Cover Perfection Tip Concealer SPF28 PA++ 6.5g""]","[1.00047, 0.596681, … 0.575912]","{'CỌ (CHỔI) QUÉT MẶT NẠ - TAY CẦM NHỰA TRONG SUỐT - TAY CẦM CÁN GỖ - CÓ THỂ SỬ DỤNG NHIỀU LẦN TIỆN LỢI 💓💓💓💓💓', 'Set 15 đầu mài giấy nhám chuyên dụng cho dân làm móng', '(GWP) Cọ trang điểm 3CE đa năng 3CE Brush Makeup Brush 15.7cm', 'Bộ 5 Đế Gắn Móng Quân Cờ Tập Vẽ Làm Nail | Đế Acrylic 5 Móng Nailbox Dễ Dàng Sử Dụng', 'Hộp 32 túi mù moi moi, rùa babythree, crybaby, mộng kuromi full box túi mù rùa moimoi quà tặng cho bé', 'Túi xách trang điểm UNMEGONI, Túi đựng mỹ phẩm dung tích lớn bằng lưới, Túi đựng đồ vệ sinh có dây kéo hai lớp di động Du lịch', 'Xịt Dưỡng Tóc Hask Curl Care Argan oil 5-in-1 Leave-in Spray 175ml Phục Hồi Tóc Khỏi Hư Tổn Mềm Mượt Tóc Hàng Úc', '[TẶNG MÚT CHE KHUYẾT ĐIỂM] MẪU MỚI - Kem Che Khuyết Điểm CKĐ The Saem Cover Perfection Tip Concealer SPF28 PA++ 6.5g'}",8
25650112647,"""(Trụ vừa) Mi trụ katun, trụ wi…","[25650112647, 11730202896, … 8270710040]","[""(Trụ vừa) Mi trụ katun, trụ wispy SEDUIRE dùng để nối mi mắt ướt đầy đủ size (dụng cụ nối mi)"", ""Bát ăn chống gù cho mèo chó thú cưng"", … ""Pink coco Son bóng màu hồng dễ thương Bề mặt gương dưỡng ẩm Dưỡng ẩm thiên nhiên lâu trôi Son môi""]","[1.000652, 0.771219, … 0.615231]","[25650112647, 25935229965, … 25472011611]","[""(Trụ vừa) Mi trụ katun, trụ wispy SEDUIRE dùng để nối mi mắt ướt đầy đủ size (dụng cụ nối mi)"", ""Bọt mềm tạo kiểu tóc xoăn Bloom 300ml"", … ""Móng chân giả NGẮN màu nude ML02 nhạt nail box chân GIÁ RẺ đẹp đơn giản thích hợp đi chơi tặng keo dán móng MC053""]","[1.000265, 0.809663, … 0.660487]","{'Pink coco Son bóng màu hồng dễ thương Bề mặt gương dưỡng ẩm Dưỡng ẩm thiên nhiên lâu trôi Son môi', 'Móng chân giả NGẮN màu nude ML02 nhạt nail box chân GIÁ RẺ đẹp đơn giản thích hợp đi chơi tặng keo dán móng MC053'}",2
10158281345,"""Mascara Chuốt Mi JUDYDOLL 2g D…","[1015

In [19]:
for n in ["item_name", "db_bi_item_name", "db_bge_item_name", "dif"]:
    print("=" * 20)
    print(df_final.to_dicts()[3][n])

Mascara Chuốt Mi JUDYDOLL 2g Dày Dài Siêu Cong Lâu Trôi Chống Thấm Nước Suốt 24h Curling Iron Mascara
['Mascara Chuốt Mi JUDYDOLL 2g Dày Dài Siêu Cong Lâu Trôi Chống Thấm Nước Suốt 24h Curling Iron Mascara', 'Thùng 48 hộp sữa lúa mạch Nestlé MILO x 110ml (12x4x110ml)', 'Combo đi sinh CB03 đầy đủ đồ dùng thiết cho mẹ và bé đi sinh-Tặng thau tắm bé,gối chống trào', 'Kem Chống Rát Da Đầu Khi Làm Hóa Chất Ziemlich | Chenglovehair, Chenglovehairs', "Kem Dưỡng Da Dưỡng Ẩm, Mềm Mịn Chống Nứt Nẻ Da Johnson's Baby 50g"]
['Mascara Chuốt Mi JUDYDOLL 2g Dày Dài Siêu Cong Lâu Trôi Chống Thấm Nước Suốt 24h Curling Iron Mascara', 'Thùng 48 hộp sữa lúa mạch Nestlé MILO x 110ml (12x4x110ml)', 'Nước tẩy trang bí đao Cocoon tẩy sạch makeup cho da dầu mụn| Cherry Beauty', 'Sáp Khử Mùi Nữ Dove Moisturizers 74gr Mỹ - Ngăn Tiết Mồ Hôi 24h', 'Gel Rửa Mặt SVR Sebiaclear Gel Moussant Hasaki Sản Phẩm Chính Hãng']
{'Gel Rửa Mặt SVR Sebiaclear Gel Moussant Hasaki Sản Phẩm Chính Hãng', 'Combo đi sinh CB03 đầy đủ đồ

In [10]:
df_final["db_bi_item_name"][0]

""
str
"""Mắt Mèo Cầu Vồng Huaxi 7 Màu T…"
"""Nấm Ngoài Da Cho Gà Đá , Chó ,…"
"""Son dưỡng môi mỡ hươu Astrid T…"
"""Thức Ăn Sấy Thú Cưng Mix Nhiều…"
"""Nước Hoa Nữ Delina Exclusif Ch…"
